In [1]:
import cv2
import numpy as np
from PIL import Image
import gradio as gr

# ✅ STEP 1: Preprocessing - Contrast Enhancement
def enhance_contrast(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))  # Adjusted contrast enhancement
    return clahe.apply(gray)

# ✅ STEP 2: Thresholding - Adaptive & Otsu
def apply_threshold(gray_image):
    _, otsu_thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    adaptive_thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY, 15, 5)  # Fine-tuned parameters
    return cv2.bitwise_and(otsu_thresh, adaptive_thresh)

# ✅ STEP 3: Tumor Segmentation - HSV Color Mask
def segment_tumor(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_tumor = np.array([0, 10, 30])   # Adjusted HSV range
    upper_tumor = np.array([30, 255, 255])
    return cv2.inRange(hsv, lower_tumor, upper_tumor)

# ✅ STEP 4: Edge Detection - Canny Filtering
def detect_edges(image):
    return cv2.Canny(image, 50, 150)  # Helps refine tumor boundary

# ✅ STEP 5: Contour Filtering - Isolate Tumor
def extract_tumor_region(binary_image, original):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    valid_contours = [cnt for cnt in contours if 200 < cv2.contourArea(cnt) < 15000]  # Adjusted range
    if valid_contours:
        largest = max(valid_contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)
        cv2.rectangle(original, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box

    return original

# ✅ STEP 6: Prediction Function
def predict_with_bbox(image):
    image = image.resize((128, 128))  # Resize for uniformity
    img_np = np.array(image)

    # Apply all processing steps
    contrast_img = enhance_contrast(img_np)
    threshold_img = apply_threshold(contrast_img)
    tumor_mask = segment_tumor(img_np)
    edge_detected = detect_edges(contrast_img)

    combined_mask = cv2.bitwise_or(threshold_img, tumor_mask)  # Merge techniques
    refined_mask = cv2.bitwise_or(combined_mask, edge_detected)  # Further refine using edges

    output_img = extract_tumor_region(refined_mask, img_np)

    return "yes" if np.any(refined_mask) else "no", output_img

# ✅ STEP 7: Gradio Interface
interface = gr.Interface(
    fn=predict_with_bbox,
    inputs=gr.Image(type='pil'),
    outputs=[
        gr.Label(label="Prediction"),
        gr.Image(label="Tumor Highlighted Image (if any)")
    ],
    title="Enhanced Brain Tumor Detector",
    description="Uses contrast boosting, thresholding, segmentation, and edge detection for improved accuracy."
)

# ✅ STEP 8: Launch App
if __name__ == "__main__":
    interface.launch()


C:\Users\haste\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\Machinelearning\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "C:\ProgramData\anaconda3\envs\Machinelearning\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\ProgramData\anaconda3\envs\Machinelearning\lib\site-packages\gradio\blocks.py", line 2147, in process_api
    result = await self.call_function(
  File "C:\ProgramData\anaconda3\envs\Machinelearning\lib\site-packages\gradio\blocks.py", line 1665, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "C:\ProgramData\anaconda3\envs\Machinelearning\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\ProgramData\anaconda3\envs\Machinelearning\lib\site-packages\anyio\_backends\_asyncio.py"